In [1]:
import threading
import queue
import time

##### Declaring the number of nodes

In [2]:
n = 12

##### Declaring Node class

In [3]:
class Node:
    def __init__(self, uid, from_prev_q, from_next_q, to_prev_q, to_next_q):
        self.uid = uid
        self.status = 'unknown'
        self._from_prev_q = from_prev_q
        self._from_next_q = from_next_q
        self._to_prev_q = to_prev_q
        self._to_next_q = to_next_q

    def send_prev(self, data):
        self._to_prev_q.put(data)

    def send_next(self, data):
        self._to_next_q.put(data)

    def accept_prev(self):
        data = self._from_prev_q.get()
        self._from_prev_q.task_done()
        return data

    def accept_next(self):
        data = self._from_next_q.get()
        self._from_next_q.task_done()
        return data

#### LCR algorithm

In [4]:
def lcr_node_worker(node):
    u = node.uid
    send = node.uid

    while True:
        node.send_next(send)

        v = node.accept_prev()
        if v == "TERM":
            break

        if v is None:
            send = v
            continue
        if v > u:
            send = v
            node.status = 'non-leader'
        elif v < u:
            send = None
        else:
            node.status = 'leader'
            return

In [5]:
def lcr_run(total_nodes):
    qs = [queue.Queue() for _ in range(total_nodes)]
    nodes = [Node(i, qs[i], None, None, qs[i - 1]) for i in range(total_nodes)]

    threads = []
    for node in nodes:
        t = threading.Thread(target=lcr_node_worker, args=(node,))
        t.start()
        threads.append(t)

    time.sleep(3)
    for q in qs:
        q.put("TERM")

    for t in threads:
        t.join()

    for node in nodes:
        print(f"{node.uid}: {node.status}")
    print("\nLCR algorithm complete.")

In [6]:
lcr_run(n)

0: non-leader
1: non-leader
2: non-leader
3: non-leader
4: non-leader
5: non-leader
6: non-leader
7: non-leader
8: non-leader
9: non-leader
10: non-leader
11: leader

LCR algorithm complete.


#### HS algorithm

In [7]:
def hs_node_worker(node):
    u = node.uid
    send_plus = (u, 'out', 1)
    send_minus = (u, 'out', 1)
    phase = 0

    while True:
        node.send_next(send_plus)
        node.send_prev(send_minus)
        send_plus, send_minus = None, None

        v_prev = node.accept_prev()
        v_next = node.accept_next()
        if v_prev == "TERM" or v_next == "TERM":
            break

        if (v_prev is not None) and v_prev[1] == 'out':
            v, out, h = v_prev
            if v > u and h > 1:
                send_plus = (v, 'out', h - 1)
            elif v > u and h == 1:
                send_minus = (v, 'in', 1)
            elif v == u:
                node.status = 'leader'

        if (v_next is not None) and v_next[1] == 'out':
            v, out, h = v_next
            if v > u and h > 1:
                send_minus = (v, 'out', h - 1)
            elif v > u and h == 1:
                send_plus = (v, 'in', 1)
            elif v == u:
                node.status = 'leader'

        if (v_prev is not None) and v_prev[1] == 'in' and v_prev[2] == 1 and v_prev[0] != u:
            v, in_, h = v_prev
            send_plus = (v, 'in', 1)
        if (v_next is not None) and v_next[1] == 'in' and v_next[2] == 1 and v_next[0] != u:
            v, in_, h = v_next
            send_minus = (v, 'in', 1)
        if v_prev == (u, 'in', 1) and v_next == (u, 'in', 1):
            phase += 1
            send_plus = (u, 'out', 2 ** phase)
            send_minus = (u, 'out', 2 ** phase)

In [8]:
def hs_run(total_nodes):
    qs1 = [queue.Queue() for _ in range(total_nodes)]
    qs2 = [queue.Queue() for _ in range(total_nodes)]
    nodes = [Node(i, qs1[i], qs2[i - 1], qs2[i], qs1[i - 1]) for i in range(total_nodes)]

    threads = []
    for node in nodes:
        t = threading.Thread(target=hs_node_worker, args=(node,))
        t.start()
        threads.append(t)

    time.sleep(3)
    for i in range(total_nodes):
        qs1[i].put("TERM")
        qs2[i].put("TERM")

    for t in threads:
        t.join()

    for node in nodes:
        print(f"{node.uid}: {node.status}")
    print("\nHS algorithm complete.")

In [9]:
hs_run(n)

0: unknown
1: unknown
2: unknown
3: unknown
4: unknown
5: unknown
6: unknown
7: unknown
8: unknown
9: unknown
10: unknown
11: leader

HS algorithm complete.
